# MENTORÍA: Predicción de indicadores sociales para países del MERCOSUR

# Práctico 3: Introducción a ML - Aprendizaje Supervisado

[Link a la descripción del proyecto](https://github.com/jfrau/DiploDatos2020/blob/master/README.md)

## Introducción

En este práctico se buscará trabajar el problema de series temporales multivariadas de esta mentoría desde el punto de vista de los algoritmos base de machine learning y Aprendizaje Supervisado. 

Sobre la base de que existen modelos específicamente diseñados para datos en forma de series temporales (los cuales escapan al alcance de los contenidos de la Diplomatura) y que tienen un desempeño bastante robusto, en este práctico nos centraremos no tanto en los resultados de los modelos base sino más bien implementar técnicas y estrategias usuales al momento de atacar problemas y que son bastante comunes en la vida diaria de un Científico de datos.

Estas estrategias se relacionan con:

- Construcción de features.
- Codificación de variables categóricas.
- Redefinición de un problema.
- Implementación y seguimiento de diferentes modelos.
- Análisis de resultados en base a los algoritmos utilizados y la cantidad de datos disponibles.


## Metodología de trabajo

En este práctico se disponibilizan datos de **BRASIL, URUGUAY y PARAGUAY** para algunos indicadores y algunos features en la franja temporal 1990-2018. Los mismos ya fueron curados, con lo cual no contienen ningún dato faltante o no nulo. Este proceso de curación se llevó a cabo teniendo en cuenta features compartidos con ARGENTINA (de la lista resultante del práctico anterior), de manera tal de poder realizar una suerte de "unión" entre los datos.

## Propuesta

Intentar explicar la variable **Índice de Gini** perteneciente al Indicador Pobreza (planteada como una columna target) utilizando las siguientes variables:

- **Brecha de pobreza a 1,90 por día (2011 PPA)**	(Indicador Pobreza)
- **Tasa de incidencia de la pobreza, sobre la base de 1,90 por día (2011 PPA) (% de la población)** (Indicador Pobreza) : porcentaje de la población que vive con menos de 1,90 dólares al día a precios internacionales de 2011.
- **Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)** (Indicador Protección Social)
- **Población activa, total** (Indicador Educación)
- **País** (Argentina, Brasil, Uruguay y Paraguay)
- **Año** (Franja temporal 1990-2018)


La propuesta se realiza en base a la posible relación o influencia de las variables de la lista anterior con el Índice de Gini (que intenta representar la inequidad social). Es importante mencionar que dicha elección tiene en cuenta además lo acotado de las bases de datos luego del proceso de curación. 

Sin embargo, dentro de los datasets que se disponibilizan existen otras variables que podrían eventualmente reemplazar a alguna de la lista o incluso a la variable target (siempre que exista una lógica en ello). *Esta es la razón por la que se plantea esta propuesta inicial, la cual puede ser modificada de acuerdo a posibles evaluaciones o decisiones del grupo.*

## Consideración importante

Como se observará más abajo el dataset que se construye contiene 28x4 (112) filas (28 años en  países sobre 4 países ) y un total de 9 columnas (sin contar la columna target) que desde el punto de vista de modelos de machine learning (sobretodo los más complejos) es considerada poca data.  Aconsejo revisar estos atículos sobre los grados de libertad en modelos de machine learning: [post1](https://machinelearningmastery.com/degrees-of-freedom-in-machine-learning/#:~:text=In%20machine%20learning%2C%20the%20degrees,a%20deep%20learning%20neural%20network.&text=In%20statistics%2C%20degrees%20of%20freedom,used%20to%20calculate%20a%20statistic.), [post2](https://discuss.analyticsvidhya.com/t/significance-of-degree-of-freedom/9127) para profundizar este tema. Esto significa que **agregar más variables y mantener solo 4 países NO es una opción**, a menos que sume otro país.

## Parte I: Preprocesamiento de los datos

In [ ]:
import pandas as pd
import io
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
#Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

**EL CODIGO QUE SIGUE ES PARA CARGAR LOS EXCEL QUE SE USAN A LA NOTEBOOK. CUANDO LO CORREN, PIDE VOLVER A CARGAR LOS ARCHIVOS. LOS QUE HAY QUE CARGAR ESTÁN EN LA CARPETA DE DRIVE**

In [ ]:
#from google.colab import files
#uploaded = files.upload()

### Carga de datos

In [ ]:

pobreza_BRA_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/pobreza_BRA_1990_2018.csv")
columns = pobreza_BRA_1990_2018.columns
pobreza_BRA_1990_2018 = pobreza_BRA_1990_2018.rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
pobreza_URU_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/pobreza_URU_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
pobreza_PAR_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/pobreza_PAR_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)

In [ ]:
# Datasets protección social
proteccionsocial_BRA_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/proteccionsocial_BRA_1990_2018.csv")
columns = proteccionsocial_BRA_1990_2018.columns
proteccionsocial_BRA_1990_2018 = proteccionsocial_BRA_1990_2018.rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
proteccionsocial_URU_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/proteccionsocial_URU_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
proteccionsocial_PAR_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/proteccionsocial_PAR_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)


In [ ]:
# Datasets educación
educacion_BRA_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/educacion_BRA_1990_2018.csv")
columns = educacion_BRA_1990_2018.columns
educacion_BRA_1990_2018 = educacion_BRA_1990_2018.rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
educacion_URU_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/educacion_URU_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)
educacion_PAR_1990_2018 = pd.read_csv("https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/Entregables/Archivos/educacion_PAR_1990_2018.csv").rename(columns = lambda x : str(x)[:4] if x in columns[4:] else x)


### Filtro por indicadores

#### Pobreza

In [ ]:
codigos_indicadores_pobreza = ['SI.POV.GINI','SI.POV.GAPS','SI.POV.DDAY']

In [ ]:
pobreza_BRA = pobreza_BRA_1990_2018[pobreza_BRA_1990_2018['Indicator Code'].isin(codigos_indicadores_pobreza)]
pobreza_URU = pobreza_URU_1990_2018[pobreza_URU_1990_2018['Indicator Code'].isin(codigos_indicadores_pobreza)]
pobreza_PAR = pobreza_PAR_1990_2018[pobreza_PAR_1990_2018['Indicator Code'].isin(codigos_indicadores_pobreza)]

In [ ]:
pobreza_PAR['Indicator Name'].unique()

#### Protección Social

In [ ]:
codigos_indicadores_proteccionsocial = ['SL.UEM.TOTL.NE.ZS']

In [ ]:
proteccionsocial_BRA = proteccionsocial_BRA_1990_2018[proteccionsocial_BRA_1990_2018['Indicator Code'].isin(codigos_indicadores_proteccionsocial)]
proteccionsocial_URU = proteccionsocial_URU_1990_2018[proteccionsocial_URU_1990_2018['Indicator Code'].isin(codigos_indicadores_proteccionsocial)]
proteccionsocial_PAR = proteccionsocial_PAR_1990_2018[proteccionsocial_PAR_1990_2018['Indicator Code'].isin(codigos_indicadores_proteccionsocial)]

In [ ]:
proteccionsocial_BRA['Indicator Name'].unique()

#### Educación

In [ ]:
codigos_indicadores_educacion = ['SL.TLF.TOTL.IN']

In [ ]:
educacion_BRA = educacion_BRA_1990_2018[educacion_BRA_1990_2018['Indicator Code'].isin(codigos_indicadores_educacion)]
educacion_URU = educacion_URU_1990_2018[educacion_URU_1990_2018['Indicator Code'].isin(codigos_indicadores_educacion)]
educacion_PAR = educacion_PAR_1990_2018[educacion_PAR_1990_2018['Indicator Code'].isin(codigos_indicadores_educacion)]

In [ ]:
educacion_BRA['Indicator Name'].unique()

### Construcción del dataset para modelos de supervisado

Primero generamos una lista con los features escogidos de cada indicador para luego formar el dataset por país con los features seleccionados de cada indicador por separado. En segundo lugar se concatenan los diferentes datasets por países para un mismo indicador. En tercer lugar, finalmente, se concatenan los diferentes datasets de los diferentes indicadores que ya contienen todos los países y los features seleccionados.

In [ ]:
variables_educacion = codigos_indicadores_educacion
variables_proteccionsocial = codigos_indicadores_proteccionsocial
variables_pobreza = codigos_indicadores_pobreza

In [ ]:
def generacion_dataset(dataset,lista_indicadores,pais_string):

    dataset = dataset[dataset['Indicator Code'].isin(lista_indicadores)]
    df = dataset.drop(['Country Name', 'Country Code', 'Indicator Code'],axis=1)
    df = df.columns.to_frame().T.append(df, ignore_index=True)
    df.columns = range(len(df.columns))
    df1 = df.T
    df1.columns = df1.iloc[0]
    df1 = df1.drop(df1.index[0])
    df1['Country'] = pais_string
    return df1

#### Variables tomadas de Educación

Generamos el dataset con los features seleccionados de educación.

In [ ]:
df_educacion_PAR = generacion_dataset(educacion_PAR,variables_educacion,'PAR')
df_educacion_BRA = generacion_dataset(educacion_BRA,variables_educacion,'BRA')
df_educacion_URU = generacion_dataset(educacion_URU,variables_educacion,'URU')

In [ ]:
educacion = pd.concat([df_educacion_PAR, df_educacion_BRA,df_educacion_URU]).rename(columns={"Indicator Name": "Year"})
educacion

Corroboramos que no existe ningún valor faltante:

In [ ]:
educacion.isna().sum()

#### Variables tomadas de Protección Social

Generamos el dataset con los features seleccionados de protección social.

In [ ]:
df_psocial_PAR = generacion_dataset(proteccionsocial_PAR,variables_proteccionsocial,'PAR')
df_psocial_BRA = generacion_dataset(proteccionsocial_BRA,variables_proteccionsocial,'BRA')
df_psocial_URU = generacion_dataset(proteccionsocial_URU,variables_proteccionsocial,'URU')

In [ ]:
psocial = pd.concat([df_psocial_PAR, df_psocial_BRA,df_psocial_URU]).rename(columns={"Indicator Name": "Year"})
psocial

Corroboramos que no existe ningún valor faltante:

In [ ]:
psocial.isna().sum()

#### Variables tomadas de Pobreza

Generamos el dataset con los features seleccionados de protección social.

In [ ]:
pobreza_PAR = generacion_dataset(pobreza_PAR,variables_pobreza,'PAR')
pobreza_BRA = generacion_dataset(pobreza_BRA,variables_pobreza,'BRA')
pobreza_URU = generacion_dataset(pobreza_URU,variables_pobreza,'URU')

In [ ]:
pobreza = pd.concat([pobreza_PAR,pobreza_BRA,pobreza_URU]).rename(columns={"Indicator Name": "Year"})
pobreza

Corroboramos que no existe ningún valor faltante:

In [ ]:
pobreza.isna().sum()

### Conformación dataset con todos los features

In [ ]:
dfmer1 = pd.merge(educacion, pobreza, on=["Country",'Year'])
df_final = dfmer1.merge(psocial,on=["Country",'Year'])
df_final

In [ ]:
#Distribución índice de Gini
df_final['Índice de Gini'].hist()

## Parte II: Preparación de los datos

### Construcción de Dataset incorporando info. de Argentina



Consigna 1: Agregar la información de ARGENTINA al dataset anterior en base a lo obtenido en el Práctico de Análisis y Curación.

Cargamos los dataset de pobreza y protección social obtenidos en el práctico anterior luego del proceso de curación. En el caso de educación, el dataset del práctico de curación no contenía el feature "Población Activa, total", por lo que lo generamos el dataset correspondiente a continuación.

In [ ]:
# Dataset pobreza
pobreza_ARG_1990_2018 = pd.read_csv(io.BytesIO(uploaded["FinalPoverty"])).rename(columns={"Features": "Indicator Name"}).drop(columns="Unnamed: 0")

# Dataset protección social
proteccionsocial_ARG_1990_2018 = pd.read_csv(io.BytesIO(uploaded["FinalSocialProtection"])).rename(columns={"Features": "Indicator Name"}).drop(columns="Unnamed: 0")

# Dataset educación
education = pd.read_csv(io.BytesIO(uploaded['educacion_mercosur_cleaned.csv'])).rename(columns={"Unnamed: 0": "Features"})
columns = education.columns
columnsDelete = columns[5:25].to_list()
columnsDelete = columnsDelete + [columns[-1]]
RelevantYearsEducation = education.drop(columnsDelete, axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
RelevantYearsEducationArg = RelevantYearsEducation[RelevantYearsEducation["Country Name"]=="Argentina"]
educacion_ARG_1990_2018 = RelevantYearsEducationArg[RelevantYearsEducationArg["Features"]=="Población activa, total"].rename(columns={"Features": "Indicator Name"})

In [ ]:
pobreza_ARG = pobreza_ARG_1990_2018[pobreza_ARG_1990_2018['Indicator Code'].isin(codigos_indicadores_pobreza)]
proteccionsocial_ARG = proteccionsocial_ARG_1990_2018[proteccionsocial_ARG_1990_2018['Indicator Code'].isin(codigos_indicadores_proteccionsocial)]
educacion_ARG = educacion_ARG_1990_2018[educacion_ARG_1990_2018['Indicator Code'].isin(codigos_indicadores_educacion)]

In [ ]:
df_pobreza_ARG = generacion_dataset(pobreza_ARG,variables_pobreza,'ARG')
df_psocial_ARG = generacion_dataset(proteccionsocial_ARG,variables_proteccionsocial,'ARG')
df_educacion_ARG = generacion_dataset(educacion_ARG,variables_educacion,'ARG')

Generamos el dataset con todos los features relevantes para Argentina:

In [ ]:
dfArgStep1 = pd.merge(df_educacion_ARG, df_pobreza_ARG, on=["Country",'Indicator Name'])
dfArgStep2 = pd.merge(dfArgStep1, df_psocial_ARG, on=["Country",'Indicator Name'])
dfArgStep3 = dfArgStep2.rename(columns={"Indicator Name": "Year"}).drop([0])
dfArgStep3

Unimos el dataset de Argentina con el dataset previo para el resto de los países, generando el dataset base a utilizar:

In [ ]:
FinalDatasetStep1 = pd.concat([df_final, dfArgStep3])
FinalDatasetStep1

In [ ]:
#Se convierte en variables númericas, las columnas correspondientes:
cols_to_num = FinalDatasetStep1.columns.tolist()
cols_to_num.remove("Year")
cols_to_num.remove("Country")
FinalDatasetStep1[cols_to_num] = FinalDatasetStep1[cols_to_num].apply(pd.to_numeric, errors='coerce', axis=1)

Se calcula la correlación entre las variables númericas para ver si existe correlación lineal entre ellas.

In [ ]:
# calculate the correlation matrix
corr = FinalDatasetStep1.corr(method ='pearson').rename(columns = lambda x : str(x)[:18])
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns, cbar=True, linewidths=0.1, annot=True)

Como puede observarse, el índice de Gini (la variable target) una relación lineal positiva con la población activa, con la brecha de pobreza y con la tasa de incidencia. Mientras que, la relación entre el desempleo total y el índice de Gini, no evidencia una relación lineal clara.

Se realizam gráficos de dispersión agrupado por país para visualizar las relaciones existentes:

In [ ]:
cols_to_num_country = cols_to_num + ["Country"]
df = FinalDatasetStep1[cols_to_num_country].rename(columns = lambda x : str(x)[:18])
cols_values = df.columns.tolist()
cols_values.remove("Country")
plt.figure(figsize=(40, 20))
sns.pairplot(data=df,
    vars=df[cols_values],hue = "Country", height=3)
plt.show()

Si se realiza un análisis de los gráficos de dispersión por país, en este caso, se reafirma que existe una relación lineal positiva entre el índice de gini y la brecha de pobreza en Brasil, Argentina y Paraguay, así como también, entre el índice de gini y la tasa de incidencia. Sin embargo, graficamente se observa que, para Uruguay, la relación entre dichas variables no es tan clara. Mientras que, la relación entre el índice de Gini y la población total activa, no es muy clara tampoco. A pesar de que la matriz de correlación muestra una relación lineal positiva, graficamente se puede observar que para BRASIL, se evidencia una relacion negativa, mientras que para el resto de los países no se puede definir una relación.

Vale destacar que el Índice de Gini muestra diferentes distribuciones según el país.

In [ ]:
df = FinalDatasetStep1.copy()
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x=df["Year"], y = df["Índice de Gini"],linewidth=3,  hue = "Country")
plt.xticks( rotation=45)
plt.show()

### Codificación/Cambios en variables


Consigna 2: convertir la columna Country en una variable dummy y codificar la columna Year

(Los siguientes links pueden ser útiles: [Link1](https://chrisalbon.com/python/data_wrangling/pandas_convert_categorical_to_dummies/), [Link2](https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159), [Link3](https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/)). 

Primero convertimos la columna country en dummy:

In [ ]:
FinalDatasetStep2 = pd.get_dummies(FinalDatasetStep1, columns = ["Country"])
FinalDatasetStep2

Ahora codificamos la columna year utilizando un uncoder ordinal debido a que trabajamos con datos que siguen un orden:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
FinalDatasetStep3 = FinalDatasetStep2.copy()
FinalDatasetStep3["Year"] = OrdinalEncoder().fit_transform(FinalDatasetStep3[['Year']])
NonNormalizedFinalDataset = FinalDatasetStep3.copy()
NonNormalizedFinalDataset = NonNormalizedFinalDataset.astype(float).round(2)
NonNormalizedFinalDataset

### Estandarización del Dataset

Consigna 3: Construir un segundo dataset con datos normalizados/estandarizados.

A continuación normalizamos con StandardScaler y MinMaxScaler en paralelo, para investigar cómo afecta cada método la distribución de nuestros datos. 

Con StandardScaler los datos no necesariamente se ubican entre 0 y 1; en este caso se espera obtener una distribución de los datos con media cero y desviación estandar 1, lo que nos serviría en caso utilicemos un algoritmo de aprendizaje que funcione mejor ante normalidad de los datos. Por su parte, con MinMaxScaler se fuerza a los datos a ubicarse entre 0 y 1 y se conserva la distribución original. Además, no se resta importancia a los valores atípicos.

######   ***Distribución original de cada columna del dataset***







In [ ]:
# Analizamos estadísticas principales
NonNormalizedFinalDataset.describe()

In [ ]:
# Funciones de Densidad

columnsDataset = NonNormalizedFinalDataset.columns
_excepciones = ['Country_ARG', 'Country_BRA', 'Country_PAR', 'Country_URU']
_lista = [x for x in columnsDataset if x not in _excepciones ]
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(20,20))
row = 0
col = 0
for x in _lista:
        sns.distplot(NonNormalizedFinalDataset[[x]], ax=axs[row, col])
        axs[row, col].set_title(x)
        col = col + 1 if col < 1 else 0
        row = row + 1 if col == 0 else row

plt.show()

######   ***Aplicación StandardScaler***










A continuación se estandarizan los datos aplicando StandardScaler. La aplicación de esta estandarización no pareciera correcta para la variable "Year", que pierde la codificación ordinal que se le había generado, y las variables dummies, que pierden su significado al asumir valores distintos a 0 y 1.
Podría pensarse en aplicar la estandarización al resto de las variables pero ya no se hablaría de un dataset normalizado con media cero y desviación estandar uno.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Definimos dataset base
DatasetToNormalize_StandardScaler = NonNormalizedFinalDataset.copy()
# Definimos columnas a normalizar
ColumnsToNormalize_StandardScaler = DatasetToNormalize_StandardScaler.columns
# Aplicamos normalización
DatasetToNormalize_StandardScaler[ColumnsToNormalize_StandardScaler] = pd.DataFrame(StandardScaler().fit_transform(DatasetToNormalize_StandardScaler[ColumnsToNormalize_StandardScaler].values), columns=ColumnsToNormalize_StandardScaler, index=DatasetToNormalize_StandardScaler.index)

NormalizedDataset_SS = DatasetToNormalize_StandardScaler
NormalizedDataset_SS


In [ ]:
# Vemos que la media para todas las variables es 0, mientras que la desviación estandar es 1
round(NormalizedDataset_SS.describe())

######  ***Aplicación MinMaxScaler***

A continuación se aplica una normalización del tipo MinMax. La normalización deja a las variables dummies inalteradas, lo que es deseable para nuestro tratamiento.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Definimos dataset base
DatasetToNormalize_MinMaxScaler = NonNormalizedFinalDataset.copy()
# Definimos columnas a normalizar
ColumnsToNormalize_MinMaxScaler = DatasetToNormalize_MinMaxScaler.columns
# Aplicamos normalización
DatasetToNormalize_MinMaxScaler[ColumnsToNormalize_MinMaxScaler] = pd.DataFrame(MinMaxScaler().fit_transform(DatasetToNormalize_MinMaxScaler[ColumnsToNormalize_MinMaxScaler].values), columns=ColumnsToNormalize_MinMaxScaler, index=DatasetToNormalize_MinMaxScaler.index)

NormalizedDataset_MMS = DatasetToNormalize_MinMaxScaler
NormalizedDataset_MMS

In [ ]:
# Vemos como los valores mínimos y máximos son 0 y 1 respectivamente.
NormalizedDataset_MMS.describe()

######   ***Comparaciones***


In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(20, 8))


ax0.set_title('Distribuciones Originales')

sns.kdeplot((NonNormalizedFinalDataset['Year']), ax=ax0)
sns.kdeplot((NonNormalizedFinalDataset['Índice de Gini']), ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Brecha de pobreza a $1,90 por día (2011 PPA) (%)'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Tasa de incidencia de la pobreza, sobre la base de $1,90 por día (2011 PPA) (% de la población)'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Country_ARG'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Country_BRA'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Country_PAR'], ax=ax0)
sns.kdeplot(NonNormalizedFinalDataset['Country_URU'], ax=ax0)
ax1.set_title('Luego de StandardScaler')

sns.kdeplot((NormalizedDataset_SS['Year']), ax=ax1)
sns.kdeplot((NormalizedDataset_SS['Índice de Gini']), ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Brecha de pobreza a $1,90 por día (2011 PPA) (%)'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Tasa de incidencia de la pobreza, sobre la base de $1,90 por día (2011 PPA) (% de la población)'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Country_ARG'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Country_BRA'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Country_PAR'], ax=ax1)
sns.kdeplot(NormalizedDataset_SS['Country_URU'], ax=ax1)
ax2.set_title('Luego de MinMaxScaler')

sns.kdeplot((NormalizedDataset_MMS['Year']), ax=ax2)
sns.kdeplot((NormalizedDataset_MMS['Índice de Gini']), ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Brecha de pobreza a $1,90 por día (2011 PPA) (%)'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Tasa de incidencia de la pobreza, sobre la base de $1,90 por día (2011 PPA) (% de la población)'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Country_ARG'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Country_BRA'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Country_PAR'], ax=ax2)
sns.kdeplot(NormalizedDataset_MMS['Country_URU'], ax=ax2)
# Nota: No se incluye "Población activa, total" al distorsionar escala.

## Parte III: Modelado

### Parte III.I


Una parte esencial del proceso de modelado (y en realidad de todo experimento científico) es llevar a cabo un *seguimiento* de todas las pruebas realizadas de manera tal de no "perder resultados" (con la consecuencia de tener que repetir experimentos ya realizados) y poder llevar un registro de que tanto se mejoró/empeoró. 

Para llevar a cabo esto disponemos de una herramienta llamada [mlflow](https://mlflow.org/) que nos permite llevar el trackeo de todo el proceso de una manera simple y rápida, además de poder guardar información que consideremos relevante. Así, la primer consigna de esta sección es **entender y setear la herramienta, instalando mlflow si es necesario, de manera tal que todo experimento que se pruebe a partir de este momento quede registrado y pueda recuperarse**. En este [link](https://github.com/PiConsulting/Pensadero/tree/master/Eventos/2020/2020-03-06_wids_mlflow) encontrarán una implementación simple con mflow de la cual pueden guiarse.

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [ ]:
import mlflow

### Parte III.II

### Separación del conjunto de datos y análisis de conjuntos resultantes

Consigna 1: Realizar la partición del dataset en conjunto de entrenamiento y test. ¿Son las distribuciones de los datos similares en ambos conjuntos? Si la respuesta es no, ¿qué decisión tomarían?.

In [ ]:
from sklearn.model_selection import train_test_split

Definimos variables target y variables independientes

In [ ]:
Target_variable ='Índice de Gini'

In [ ]:
X_variables = NonNormalizedFinalDataset.columns.tolist()
X_variables.remove(Target_variable)


In [ ]:
X = NonNormalizedFinalDataset[X_variables]
                               
y = NonNormalizedFinalDataset[Target_variable]

In [ ]:
# X: Split data into train and test
X_train = X[:-30]
X_test = X[-30:]

In [ ]:
# Target: Split into training/testing sets
y_train = y[:-30]
y_test = y[-30:]

In [ ]:
print("Dimensión X_train:" + str(X_train.shape))
print("Dimensión X_test:" + str(X_test.shape))
print("Dimensión y_train:" + str(y_train.shape))
print("Dimensión y_test:" + str(y_test.shape))

In [ ]:
X_train.dtypes

A continuación se analiza la distribución de los datos por conjunto. No se observan grandes diferencias en las funciones de densidad.

In [ ]:
# Variable dependiente

plt.figure(figsize=(10,6))
sns.distplot(y_train, label="Train")
sns.distplot(y_test, label="Test")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Distribución de la variable dependiente")

In [ ]:
# Variables explicativas

columns = X_train.columns

fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(20,20))
row = 0
col = 0
for columns in columns:
        sns.distplot(X_train[[columns]], ax=axs[row, col], label="train")
        sns.distplot(X_test[[columns]], ax=axs[row, col], label="test")
        axs[row, col].set_title(columns)
        col = col + 1 if col < 1 else 0
        row = row + 1 if col == 0 else row

plt.show()

Como puede observarse en el gráfico anterior, las distribuciones de los valores de X train y test, son similares. Indicando que es una división correcta.

### Regresión Lineal y Polinomial

Consigna 2: Probar modelos de *regresión lineal* y *polinomial* (elegir al menos dos grados diferentes) sobre los **datos sin normalizar**. ¿Cómo son los errores sobre el conjunto de entrenamiento y sobre el conjunto de test? Analizar los resultados.

#### Regresión Lineal

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [ ]:
mlflow.set_experiment("Linear Regression On Non Normalized Dataset")

with mlflow.start_run():

  # Entrenamiento:
  linear_regression_NonNormalized = linear_model.LinearRegression()
  linear_regression_NonNormalized.fit(X_train, y_train)

  # Predicción:
  y_train_pred_LinearRegressionNonNormalized = linear_regression_NonNormalized.predict(X_train)
  y_test_pred_LinearRegressionNonNormalized = linear_regression_NonNormalized.predict(X_test)
  
  # Evaluación:
  train_error_LinearRegressionNonNormalized = mean_squared_error(y_train, y_train_pred_LinearRegressionNonNormalized)
  test_error_LinearRegressionNonNormalized = mean_squared_error(y_test, y_test_pred_LinearRegressionNonNormalized)
  train_R2_LinearRegressionNonNormalized = r2_score(y_train, y_train_pred_LinearRegressionNonNormalized)
  test_R2_LinearRegressionNonNormalized = r2_score(y_test, y_test_pred_LinearRegressionNonNormalized)

  mlflow.log_metric("Train Error", train_error_LinearRegressionNonNormalized)
  mlflow.log_metric("Test Error", test_error_LinearRegressionNonNormalized)
  mlflow.log_metric("Train R2", train_R2_LinearRegressionNonNormalized)
  mlflow.log_metric("Test R2", test_R2_LinearRegressionNonNormalized)
  mlflow.sklearn.log_model(linear_regression_NonNormalized, "Linear Regression On Non Normalized Dataset")

In [ ]:
# Verificamos que la variable dependiente se encuentre entre cero y 100
_check_train = (0 > y_train_pred_LinearRegressionNonNormalized) | (y_train_pred_LinearRegressionNonNormalized>100)
_check_train = _check_train.sum()
print(f"Existen {_check_train} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X train.")

_check_test= (0 > y_test_pred_LinearRegressionNonNormalized) | (y_test_pred_LinearRegressionNonNormalized>100)
_check_test = _check_test.sum()
print(f"Existen {_check_test} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X test.")

In [ ]:
print("El error cuadrático medio en train es:" + str(train_error_LinearRegressionNonNormalized))
print("El error cuadrático medio en test es:" + str(test_error_LinearRegressionNonNormalized))
print("El R2 en train es:" + str(train_R2_LinearRegressionNonNormalized))
print("El R2 en test es:" + str(test_R2_LinearRegressionNonNormalized))

In [ ]:
#Modelo completo
Reg_model = np.concatenate((y_train_pred_LinearRegressionNonNormalized, y_test_pred_LinearRegressionNonNormalized), axis=0)

In [ ]:
# Plot outputs

plt.figure(figsize=(10, 6))
plt.plot(y, color='green', linewidth=2, label = "Actual")  
plt.plot(Reg_model, color='blue', linewidth=2, label = "Predicted")
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")

plt.show()

In [ ]:
# The coefficients
print('Coefficients: \n', linear_regression_NonNormalized.coef_)

Grafico solo de train:

In [ ]:
# Plot outputs
plt.plot(y_train, color='green', linewidth=3, label = "Train Actual")  
plt.plot(y_train_pred_LinearRegressionNonNormalized, color='blue', linewidth=3, label = "Train Predicted")
plt.xticks(())
plt.yticks(())

plt.legend(loc = "Left corner")

plt.show()

Graficos solo de test:

In [ ]:
# Plot outputs
plt.plot(y_test, color='green', linewidth=3, label = "Test Actual")  
plt.plot(y_test_pred_LinearRegressionNonNormalized, color='blue', linewidth=3, label = "Test Predicted")
plt.xticks(())
plt.yticks(())

plt.legend(loc = "Left corner")

plt.show()

#### Regresión Polinomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
train_errors_PolynomialRegressionNonNormalized = []
test_errors_PolynomialRegressionNonNormalized = []
train_R2s_PolynomialRegressionNonNormalized = []
test_R2s_PolynomialRegressionNonNormalized = []

degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9]

for degree in degrees:

  # Entrenamiento:
  polynomial_regression_NonNormalized = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression(fit_intercept = True))
  polynomial_regression_NonNormalized.fit(X_train, y_train)

  # Predicción:
  y_train_pred_PolynomialRegressionNonNormalized = polynomial_regression_NonNormalized.predict(X_train)
  y_test_pred_PolynomialRegressionNonNormalized = polynomial_regression_NonNormalized.predict(X_test)

  # Evaluación:
  train_error_PolynomialRegressionNonNormalized = mean_squared_error(y_train, y_train_pred_PolynomialRegressionNonNormalized)
  test_error_PolynomialRegressionNonNormalized = mean_squared_error(y_test, y_test_pred_PolynomialRegressionNonNormalized)
  train_errors_PolynomialRegressionNonNormalized.append(train_error_PolynomialRegressionNonNormalized)
  test_errors_PolynomialRegressionNonNormalized.append(test_error_PolynomialRegressionNonNormalized)
  train_R2_PolynomialRegressionNonNormalized = r2_score(y_train, y_train_pred_PolynomialRegressionNonNormalized)
  test_R2_PolynomialRegressionNonNormalized = r2_score(y_test, y_test_pred_PolynomialRegressionNonNormalized)
  train_R2s_PolynomialRegressionNonNormalized.append(train_R2_PolynomialRegressionNonNormalized)
  test_R2s_PolynomialRegressionNonNormalized.append(test_R2_PolynomialRegressionNonNormalized)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(degrees, train_errors_PolynomialRegressionNonNormalized, linewidth=3,  label="train")
plt.plot(degrees, test_errors_PolynomialRegressionNonNormalized,  linewidth=3, label="test")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylabel("error")
plt.xlabel("degree")
plt.title("Error cuadrático medio para diferentes grados del polinomio")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(degrees, train_R2s_PolynomialRegressionNonNormalized,linewidth=3, label="train")
plt.plot(degrees, test_R2s_PolynomialRegressionNonNormalized,linewidth=3, label="test")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylabel("error")
plt.xlabel("degree")
plt.title("R2 para diferentes grados del polinomio")
plt.show()

In [ ]:
PolynomialRegressionComparisonsDataframe = pd.DataFrame(data = [train_errors_PolynomialRegressionNonNormalized,
                                                      test_errors_PolynomialRegressionNonNormalized,
                                                      train_R2s_PolynomialRegressionNonNormalized,
                                                      test_R2s_PolynomialRegressionNonNormalized], 
                                              columns = degrees ,
                                              index = ["Train MSE", "Test MSE", "Train R2", "Test R2"])

PolynomialRegressionComparisonsDataframe

De los gráficos y tablas anteriores se desprende que el menor error cuadrático medio para test y el mayor R2 para test se da para grado 2 del polinomio.


A continuación nos quedamos con el modelo de grado de polinomio 2 y graficamos:

In [ ]:
# Entrenamiento:
polynomial_regression_NonNormalizedFinal = make_pipeline(PolynomialFeatures(2), linear_model.LinearRegression(fit_intercept = True))
polynomial_regression_NonNormalizedFinal.fit(X_train, y_train)

# Predicción:
y_train_pred_PolynomialRegressionNonNormalizedFinal = polynomial_regression_NonNormalizedFinal.predict(X_train)
y_test_pred_PolynomialRegressionNonNormalizedFinal = polynomial_regression_NonNormalizedFinal.predict(X_test)

In [ ]:
# Verificamos que la variable dependiente se encuentre entre cero y 100
_check_train = (0 > y_train_pred_PolynomialRegressionNonNormalizedFinal) | (y_train_pred_PolynomialRegressionNonNormalizedFinal>100)
_check_train = _check_train.sum()
print(f"Existen {_check_train} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X train.")

_check_test= (0 > y_test_pred_PolynomialRegressionNonNormalizedFinal) | (y_test_pred_PolynomialRegressionNonNormalizedFinal>100)
_check_test = _check_test.sum()
print(f"Existen {_check_test} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X test.")

In [ ]:
#Modelo completo
Pol_model = np.concatenate((y_train_pred_PolynomialRegressionNonNormalizedFinal, y_test_pred_PolynomialRegressionNonNormalizedFinal), axis=0)

In [ ]:
# Plot outputs

plt.figure(figsize=(10, 6))
plt.plot(y, color='green', linewidth=3, label = "Actual")  
plt.plot(Pol_model, color='blue', linewidth=2, label = "Predicted")
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")

Solo con los datos de train:

In [ ]:
# Plot outputs
plt.plot(y_train, color='green', linewidth=3, label = "Train")  
plt.plot(y_train_pred_PolynomialRegressionNonNormalizedFinal, color='red', linewidth=3, label = "Predicted")
plt.xticks(())
plt.yticks(())

plt.legend(loc = "Left corner")

plt.show()

Solo con los datos de test:

In [ ]:
# Plot outputs
plt.plot(y_test, color='green', linewidth=3, label = "Test")  
plt.plot(y_test_pred_PolynomialRegressionNonNormalizedFinal, color='red', linewidth=3, label = "Predicted")
plt.xticks(())
plt.yticks(())

plt.legend(loc = "Left corner")

plt.show()

### Arbol de decisión/Random Forest

Consigna 3: Probar algún modelo del tipo *árbol de decisión/random forest* para problemas de regresión también sobre los **datos sin normalizar**. Extraer y estudiar el feature importance. Análizar la performance de estos modelos y comparar con los resultados del punto 2 (**Sabemos que esto va a dar muy mal por la poca cantidad de datos!**).

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
mlflow.set_experiment("Decision Tree Regressor On Non Normalized Dataset")

with mlflow.start_run():

  # Entrenamiento:
  clf_NonNormalized = DecisionTreeRegressor()
  clf_NonNormalized.fit(X_train, y_train)

  # Predicción:
  y_train_pred_ClfNonNormalized = clf_NonNormalized.predict(X_train)  
  y_test_pred_ClfNonNormalized = clf_NonNormalized.predict(X_test)
  
  # Evaluación:
  train_error_ClfNonNormalized = mean_squared_error(y_train, y_train_pred_ClfNonNormalized)
  test_error_ClfNonNormalized = mean_squared_error(y_test, y_test_pred_ClfNonNormalized)

  mlflow.log_metric("Train Error", train_error_ClfNonNormalized)
  mlflow.log_metric("Test Error", test_error_ClfNonNormalized)
  mlflow.sklearn.log_model(clf_NonNormalized, "Decision Tree Regressor On Non Normalized Dataset")

In [ ]:
# Verificamos que la variable dependiente se encuentre entre cero y 100
_check_train = (0 > y_train_pred_ClfNonNormalized) | (y_train_pred_ClfNonNormalized>100)
_check_train = _check_train.sum()
print(f"Existen {_check_train} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X train.")

_check_test= (0 > y_test_pred_ClfNonNormalized) | (y_test_pred_ClfNonNormalized>100)
_check_test = _check_test.sum()
print(f"Existen {_check_test} valores predichos por el modelo que están fuera del rango [0,100] cuando se utiliza X test.")

In [ ]:
print("El error cuadrático medio en train es:" + str(train_error_ClfNonNormalized))
print("El error cuadrático medio en test es:" + str(test_error_ClfNonNormalized))

### Modelaje sobre dataset normalizado

Consigna 4: Repetir los pasos 2 y 3 sobre el **dataset normalizado**: ¿Cambian los resultados? ¿Se observa alguna diferencia en performance? ¿Algo para destacar?.

#### Regresión Lineal y Polinomial sobre Dataset Normalizado

In [ ]:
Target_nom_variable = 'Índice de Gini'

In [ ]:
X_norm_variables = NormalizedDataset_MMS.columns.tolist()
X_norm_variables.remove(Target_nom_variable)

In [ ]:
X_norm = NormalizedDataset_MMS[X_norm_variables]

y_norm = NormalizedDataset_MMS[Target_nom_variable]

In [ ]:
# X: Split data into train and test
X_norm_train = X_norm[:-30]
X_norm_test = X_norm[-30:]

In [ ]:
# Target: Split into training/testing sets
y_norm_train = y_norm[:-30]
y_norm_test = y_norm[-30:]


##### Regresión Lineal

In [ ]:
mlflow.set_experiment("Linear Regression On Normalized Dataset")

with mlflow.start_run():

  # Entrenamiento:
  linear_regression_Normalized = linear_model.LinearRegression()
  linear_regression_Normalized.fit(X_norm_train, y_norm_train)

  # Predicción:
  y_train_pred_LinearRegressionNormalized = linear_regression_Normalized.predict(X_norm_train)
  y_test_pred_LinearRegressionNormalized = linear_regression_Normalized.predict(X_norm_test)
  
  # Evaluación:
  train_error_LinearRegressionNormalized = mean_squared_error(y_norm_train, y_train_pred_LinearRegressionNormalized)
  test_error_LinearRegressionNormalized = mean_squared_error(y_norm_test, y_test_pred_LinearRegressionNormalized)
  train_R2_LinearRegressionNormalized = r2_score(y_norm_train, y_train_pred_LinearRegressionNormalized)
  test_R2_LinearRegressionNormalized = r2_score(y_norm_test, y_test_pred_LinearRegressionNormalized)

  mlflow.log_metric("Train Error", train_error_LinearRegressionNormalized)
  mlflow.log_metric("Test Error", test_error_LinearRegressionNormalized)
  mlflow.log_metric("Train R2", train_R2_LinearRegressionNormalized)
  mlflow.log_metric("Test R2", test_R2_LinearRegressionNormalized)
  mlflow.sklearn.log_model(linear_regression_Normalized, "Linear Regression On Normalized Dataset")

In [ ]:
print("El error cuadrático medio en train es:" + str(train_error_LinearRegressionNormalized))
print("El error cuadrático medio en test es:" + str(test_error_LinearRegressionNormalized))
print("El R2 en train es:" + str(train_R2_LinearRegressionNormalized))
print("El R2 en test es:" + str(test_R2_LinearRegressionNormalized))

In [ ]:
# Plot outputs   
plt.plot( y_train_pred_LinearRegressionNormalized, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm_train, color='red', linewidth=3, label = "Train") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

In [ ]:
# Plot outputs   
plt.plot( y_test_pred_LinearRegressionNormalized, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm_test, color='red', linewidth=3, label = "Test") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

In [ ]:
#Modelo completo
reg_norm_model = np.concatenate((y_train_pred_LinearRegressionNormalized, y_test_pred_LinearRegressionNormalized), axis=0)

In [ ]:
# Plot outputs   
plt.plot( reg_norm_model, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm, color='red', linewidth=3, label = "Actual") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

##### Regresión Polinomial

In [ ]:
train_errors_PolynomialRegressionNormalized = []
test_errors_PolynomialRegressionNormalized = []
train_R2s_PolynomialRegressionNormalized = []
test_R2s_PolynomialRegressionNormalized = []

degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9]

for degree in degrees:

  # Entrenamiento:
  polynomial_regression_Normalized = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
  polynomial_regression_Normalized.fit(X_norm_train, y_norm_train)

  # Predicción:
  y_train_pred_PolynomialRegressionNormalized = polynomial_regression_Normalized.predict(X_norm_train)
  y_test_pred_PolynomialRegressionNormalized = polynomial_regression_Normalized.predict(X_norm_test)

  # Evaluación:
  train_error_PolynomialRegressionNormalized = mean_squared_error(y_norm_train, y_train_pred_PolynomialRegressionNormalized)
  test_error_PolynomialRegressionNormalized = mean_squared_error(y_norm_test, y_test_pred_PolynomialRegressionNormalized)
  train_errors_PolynomialRegressionNormalized.append(train_error_PolynomialRegressionNormalized)
  test_errors_PolynomialRegressionNormalized.append(test_error_PolynomialRegressionNormalized)
  train_R2_PolynomialRegressionNormalized = r2_score(y_norm_train, y_train_pred_PolynomialRegressionNormalized)
  test_R2_PolynomialRegressionNormalized = r2_score(y_norm_test, y_test_pred_PolynomialRegressionNormalized)
  train_R2s_PolynomialRegressionNormalized.append(train_R2_PolynomialRegressionNormalized)
  test_R2s_PolynomialRegressionNormalized.append(test_R2_PolynomialRegressionNormalized)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(degrees, train_errors_PolynomialRegressionNormalized, linewidth = 3, label="train")
plt.plot(degrees, test_errors_PolynomialRegressionNormalized, linewidth = 3, label="test")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylabel("error")
plt.xlabel("degree")
plt.title("Error cuadrático medio para diferentes grados del polinomio")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(degrees, train_R2s_PolynomialRegressionNormalized,linewidth = 3, label="train")
plt.plot(degrees, test_R2s_PolynomialRegressionNormalized,linewidth = 3, label="test")
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylabel("error")
plt.xlabel("degree")
plt.title("R2 para diferentes grados del polinomio")
plt.show()

In [ ]:
PolynomialRegressionComparisonsDataframeNorm = pd.DataFrame(data = [train_errors_PolynomialRegressionNormalized,
                                                                    test_errors_PolynomialRegressionNormalized,
                                                                    train_R2s_PolynomialRegressionNormalized,
                                                                    test_R2s_PolynomialRegressionNormalized], 
                                                            columns = degrees ,
                                                            index = ["Train MSE", "Test MSE", "Train R2", "Test R2"])

PolynomialRegressionComparisonsDataframeNorm

In [ ]:
# Plot outputs   
plt.plot( y_train_pred_PolynomialRegressionNormalized, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm_train, color='red', linewidth=3, label = "Train") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

In [ ]:
# Plot outputs   
plt.plot( y_test_pred_PolynomialRegressionNormalized, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm_test, color='red', linewidth=3, label = "Test") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

In [ ]:
#Modelo completo
pol_norm_model = np.concatenate((y_train_pred_PolynomialRegressionNormalized, y_test_pred_PolynomialRegressionNormalized), axis=0)

In [ ]:
# Plot outputs   
plt.plot( pol_norm_model, color='blue', linewidth=3, label = "Predicted") 
plt.plot(y_norm, color='red', linewidth=3, label = "Test") 
plt.xticks(())
plt.yticks(())
plt.legend(loc = "Left corner")
plt.show()

#### Arbol de Decisión sobre Dataset Normalizado

In [ ]:
mlflow.set_experiment("Decision Tree Regressor On Normalized Dataset")

with mlflow.start_run():

  # Entrenamiento:
  clf_Normalized = DecisionTreeRegressor()
  clf_Normalized.fit(X_norm_train, y_norm_train)

  # Predicción:
  y_train_pred_ClfNormalized = clf_Normalized.predict(X_norm_train)  
  y_test_pred_ClfNormalized = clf_Normalized.predict(X_norm_test)
  
  # Evaluación:
  train_error_ClfNormalized = mean_squared_error(y_norm_train, y_train_pred_ClfNormalized)
  test_error_ClfNormalized = mean_squared_error(y_norm_test, y_test_pred_ClfNormalized)

  mlflow.log_metric("Train Error", train_error_ClfNormalized)
  mlflow.log_metric("Test Error", test_error_ClfNormalized)
  mlflow.sklearn.log_model(clf_Normalized, "Decision Tree Regressor On Normalized Dataset")

In [ ]:
print("El error cuadrático medio en train es:" + str(train_error_ClfNormalized))
print("El error cuadrático medio en test es:" + str(test_error_ClfNormalized))

### Práctica de MLflow

Consigna 5: A modo de práctica recuperar algún resultado usando mlflow. Nota: Se recomienda guardar las métricas obtenidas, modelos, tipo de algoritmo probado (y por qué no los dataset usados). 

In [ ]:
mlflow.search_runs()

## Conclusiones

Orientar conclusiones a:

Analisis de problemas al momento de modelar un dataset reducido (¿Qué rol cumple el overfitting en este caso?)